<a href="https://colab.research.google.com/github/GiX007/auto-suggest-offline/blob/main/src/tutorials/toy_recommendation_workflow_simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End-to-End Toy Simulation of Auto-Suggest Recommendation Tasks

This notebook provides a complete walkthrough of the Auto-Suggest operator prediction pipeline as described in the original paper, using simplified synthetic sequences and the Titanic dataset to simulate a realistic workflow.

We cover:
*   **Single-Operator Feature Extraction**
  For each core pandas operations (```merge```, ```groupby```, ```pivot```, ```melt```), we extract features that characterize its input table and train individual binary classifiers.
*   **Single-Operator Prediction Models**
  Each operator gets its own predictor trained on synthetic input-label pairs, to estimate whether a given table is suitable for that operation.
*   **Next Operator Prediction**
  We simulate notebook execution sequences and predict the next operator using **n-gram models** for context-based predictions and **RNN models (LSTM)** for learning sequence patterns over time.
*   **Combining Sequence Context with Table Characteristics**
  In the final stage, we combine **RNN output** (sequence context vector) and **Single-operator model scores** (e.g. $P(groupby | Ti)$, $P(pivot | Ti)$, $P(melt | Ti)$, $P(merge | Ti)$) into a unified **MLP classifier** that predicts the next likely operator

This notebook uses toy-scale examples to understand the design logic, feature extraction pipeline, and modeling approach behind operator recommendation. Scaling to real notebooks (e.g., 100K examples or NP-hard CMUT grouping) requires additional infrastructure and optimization.

**Note:** The implementations in this notebook are soft approximations and not exact replicas of those described in the paper, particularly for optimization-based components like AMPT and CMUT, which are simplified to enable faster experimentation and easier debugging.

## Single Prediction Task

### Join Features Extraction

In [ ]:
import pandas as pd
titanic_df = pd.read_csv('titanic.csv')

In [ ]:
# name, sex, embarked cols to a new dataframe
survived_left = titanic_df.loc[titanic_df['Survived'] == 1, ['PassengerId', 'Name', 'Sex', 'Age', 'Embarked']]
print(survived_left.shape)
survived_left.head()

(342, 5)


,PassengerId,Name,Sex,Age,Embarked
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,C
2,3,"Heikkinen, Miss. Laina",female,26.0,S
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,S
8,9,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,S
9,10,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,C


In [ ]:
survived_right = titanic_df.loc[titanic_df['Survived'] == 1, ['PassengerId', 'Pclass', 'Ticket', 'Fare', 'Cabin', 'Embarked']]
print(survived_right.shape)
survived_right.head()

(342, 6)


,PassengerId,Pclass,Ticket,Fare,Cabin,Embarked
1,2,1,PC 17599,71.2833,C85,C
2,3,3,STON/O2. 3101282,7.9250,NaN,S
3,4,1,113803,53.1000,C123,S
8,9,3,347742,11.1333,NaN,S
9,10,2,237736,30.0708,NaN,C


In [ ]:
# example of join operation
merged_df = pd.merge(survived_left, survived_right, on='PassengerId')
print(merged_df.shape)
merged_df.head()

(342, 10)


,PassengerId,Name,Sex,Age,Embarked_x,Pclass,Ticket,Fare,Cabin,Embarked_y
0,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,C,1,PC 17599,71.2833,C85,C
1,3,"Heikkinen, Miss. Laina",female,26.0,S,3,STON/O2. 3101282,7.9250,NaN,S
2,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,S,1,113803,53.1000,C123,S
3,9,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,S,3,347742,11.1333,NaN,S
4,10,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,C,2,237736,30.0708,NaN,C


In [ ]:
# Feature 1: Distinct-value-ratio (distinct tuples over total number of tuples)
# Key columns typically have ratio close to 1 (unique values)

# drop duplicates from both tables (if any)
survived_left = survived_left.drop_duplicates()
survived_right = survived_right.drop_duplicates()

left_distinct_tuples = survived_left.shape[0]
right_distinct_tuples = survived_right.shape[0]

left_distinct_tuples_ratio = left_distinct_tuples / (left_distinct_tuples)
right_distinct_tuples_ratio = right_distinct_tuples / (right_distinct_tuples)

print(left_distinct_tuples_ratio)
print(right_distinct_tuples_ratio)

1.0
1.0


In [ ]:
# Feature 2: Value-overlap (measures how much the values in the two column sets overlap)
# Good join candidates typically have high overlap

# calculate intersection, union and jaccard
intersection = len(set(survived_left['Embarked']).intersection(set(survived_right['Embarked'])))
union = len(set(survived_left['Embarked']).union(set(survived_right['Embarked'])))
jaccard = intersection / union
print(jaccard)

left_to_right_containment = intersection / left_distinct_tuples
right_to_left_containment = intersection / right_distinct_tuples

print(left_to_right_containment)
print(right_to_left_containment)

1.0
0.011695906432748537
0.011695906432748537


In [ ]:
# Feature 3: Value-range-overlap (for numeric columns - checks if numeric ranges overlap)
# If the range overlap is low, columns may not be good candidates to join
# e.g. S=[5,10] and S'=[8,15], then inter=2 and union=10, so range overlap=0.2
# This helps to identify not to include as candidates columns with different ranges, e.g. left with 0-100 and right with 0-10 range values

for col_1 in survived_left.columns:
  for col_2 in survived_right.columns:
    # if they are cols with numeric values
    if pd.api.types.is_numeric_dtype(survived_left[col_1]) and pd.api.types.is_numeric_dtype(survived_right[col_2]):
      range_1 = (survived_left[col_1].min(), survived_left[col_1].max())
      range_2 = (survived_right[col_2].min(), survived_right[col_2].max())
      intersection = max(0, min(range_1[1], range_2[1]) - max(range_1[0], range_2[0])) # max(0,min(max(S),max(S′))−max(min(S),min(S′)))
      union = max(range_1[1], range_2[1]) - min(range_1[0], range_2[0]) # max(max(S),max(S′))−min(min(S),min(S′))
      value_range_overlap = intersection / union if union > 0 else 0
      print(f"Value range overlap between {col_1} and {col_2}: {value_range_overlap}")

Value range overlap between PassengerId and PassengerId: 1.0
Value range overlap between PassengerId and Pclass: 0.0011248593925759281
Value range overlap between PassengerId and Fare: 0.573403595505618
Value range overlap between Age and PassengerId: 0.08768182737921265
Value range overlap between Age and Pclass: 0.025131942699170646
Value range overlap between Age and Fare: 0.15532981528282985


In [ ]:
# Feature 4: Column-value-types (compares data types of columns)
# Columns with the same data type are more likely to be joined, but
# two string columns with high overlap are better join candidates than numeric columns

data_type_match = {}
for col_1 in survived_left.columns:
  for col_2 in survived_right.columns:
    # check if they are of the same data type
    if survived_left[col_1].dtype == survived_right[col_2].dtype:
      data_type_match[(col_1, col_2)] = True
    else:
      data_type_match[(col_1, col_2)] = False

for key, value in data_type_match.items():
  print(f"{key}: {value}")

('PassengerId', 'PassengerId'): True
('PassengerId', 'Pclass'): True
('PassengerId', 'Ticket'): False
('PassengerId', 'Fare'): False
('PassengerId', 'Cabin'): False
('PassengerId', 'Embarked'): False
('Name', 'PassengerId'): False
('Name', 'Pclass'): False
('Name', 'Ticket'): True
('Name', 'Fare'): False
('Name', 'Cabin'): True
('Name', 'Embarked'): True
('Sex', 'PassengerId'): False
('Sex', 'Pclass'): False
('Sex', 'Ticket'): True
('Sex', 'Fare'): False
('Sex', 'Cabin'): True
('Sex', 'Embarked'): True
('Age', 'PassengerId'): False
('Age', 'Pclass'): False
('Age', 'Ticket'): False
('Age', 'Fare'): True
('Age', 'Cabin'): False
('Age', 'Embarked'): False
('Embarked', 'PassengerId'): False
('Embarked', 'Pclass'): False
('Embarked', 'Ticket'): True
('Embarked', 'Fare'): False
('Embarked', 'Cabin'): True
('Embarked', 'Embarked'): True


In [ ]:
# Feature 5: Left-ness (position in table)
# Columns to the left of tables are more likely to be join columns

for col in survived_left.columns:
  leftness = survived_left.columns.get_loc(col) / (survived_left.shape[1])
  print(f"{col}: {leftness}")

print(" ")
for col in survived_right.columns:
  leftness = survived_right.columns.get_loc(col) / (survived_right.shape[1])
  print(f"{col}: {leftness}")

PassengerId: 0.0
Name: 0.2
Sex: 0.4
Age: 0.6
Embarked: 0.8
 
PassengerId: 0.0
Pclass: 0.16666666666666666
Ticket: 0.3333333333333333
Fare: 0.5
Cabin: 0.6666666666666666
Embarked: 0.8333333333333334


In [ ]:
# Feature 6: Sorted-ness (sorted columns are more likely key columns)

# Check sortedness for each column
for col in survived_left.columns:
  is_sorted = survived_left[col].is_monotonic_increasing
  print(f"Column '{col}' is sorted: {is_sorted}")

print(" ")
for col in survived_right.columns:
  is_sorted = survived_right[col].is_monotonic_increasing
  print(f"Column '{col}' is sorted: {is_sorted}")

Column 'PassengerId' is sorted: True
Column 'Name' is sorted: False
Column 'Sex' is sorted: False
Column 'Age' is sorted: False
Column 'Embarked' is sorted: False
 
Column 'PassengerId' is sorted: True
Column 'Pclass' is sorted: False
Column 'Ticket' is sorted: False
Column 'Fare' is sorted: False
Column 'Cabin' is sorted: False
Column 'Embarked' is sorted: False


In [ ]:
# Feature 7: Single-column-candidate
# Indicates whether a candidate is a single-column or not
# Single-column joins are more common and may be preferred

# suppose we have some join candidates
candidates = ["PassengerId", ["Sex", "Age"], "Embarked"]

for candidate in candidates:
  if isinstance(candidate, str):
    print(f"{candidate} is a single-column candidate")
    single_column_candidate = True
  else:
    print(f"{candidate} is a multi-column candidate")
    single_column_candidate = False

PassengerId is a single-column candidate
['Sex', 'Age'] is a multi-column candidate
Embarked is a single-column candidate


In [ ]:
# Feature 8: Table-level-statistics - Information about the tables (rows)
# Helps assess the reliability of overlap metrics in conjuction with other features

survived_left_rows = survived_left.shape[0]
survived_right_rows = survived_right.shape[0]
rows_count_ratio = survived_left_rows / survived_right_rows
print(rows_count_ratio)

1.0


### Join Model Training and Evaluation

In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Define candidate pairs (manually for this tutorial)
# In a real pipeline, all possible column pairs between left and right tables would be enumerated (including single- and multi-column combinations),
# and labeled based on whether a join between them successfully matches rows and datatypes (at least). For now, we manually define a small number of
# candidate pairs for illustration. Each tuple contains: (left_col, right_col, label) where label = 1 if it's a good join (true positive), otherwise 0.

candidate_pairs = [
    ("PassengerId", "PassengerId", 1),  # good join
    ("Embarked", "Embarked", 0),        # weak join
    ("Sex", "Pclass", 0),               # different types
    ("Age", "Fare", 0),                 # numeric mismatch
    ("Embarked", "Cabin", 0),           # unlikely
]

In [ ]:
# Join Feature extraction function
def extract_join_features(col1, col2):
    features = {}

    # Feature 1: Distinct-value-ratio
    features['left_distinct_ratio'] = survived_left[col1].nunique() / len(survived_left)
    features['right_distinct_ratio'] = survived_right[col2].nunique() / len(survived_right)

    # Feature 2: Jaccard overlap
    set1, set2 = set(survived_left[col1].dropna()), set(survived_right[col2].dropna())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2)) or 1
    features['jaccard_overlap'] = intersection / union
    features['left_containment'] = intersection / (len(set1) or 1)
    features['right_containment'] = intersection / (len(set2) or 1)

    # Feature 3: Range overlap (only if numeric)
    if pd.api.types.is_numeric_dtype(survived_left[col1]) and pd.api.types.is_numeric_dtype(survived_right[col2]):
        range1 = (survived_left[col1].min(), survived_left[col1].max())
        range2 = (survived_right[col2].min(), survived_right[col2].max())
        inter = max(0, min(range1[1], range2[1]) - max(range1[0], range2[0]))
        union_range = max(range1[1], range2[1]) - min(range1[0], range2[0])
        features['value_range_overlap'] = inter / union_range if union_range > 0 else 0
    else:
        features['value_range_overlap'] = 0

    # Feature 4: Same data type
    features['same_dtype'] = int(survived_left[col1].dtype == survived_right[col2].dtype)

    # Feature 5: Leftness
    features['left_leftness'] = survived_left.columns.get_loc(col1) / len(survived_left.columns)
    features['right_leftness'] = survived_right.columns.get_loc(col2) / len(survived_right.columns)

    # Feature 6: Sortedness
    features['left_sorted'] = int(survived_left[col1].is_monotonic_increasing)
    features['right_sorted'] = int(survived_right[col2].is_monotonic_increasing)

    # Feature 7: Single-column-candidate (always 1 here since all are single columns)
    features['single_col'] = 1

    # Feature 8: Table-level row ratio
    features['row_ratio'] = len(survived_left) / len(survived_right)

    return features

In [ ]:
# Build dataset
X = []
y = []
columns = None

for col1, col2, label in candidate_pairs:
  feats = extract_join_features(col1, col2) # feats is a dictionary with all features
  if columns is None:
    columns = list(feats.keys())  # just create a list of feataure names (feats.keys()) on the first iter
  X.append(list(feats.values()))  # appends the feature values for each pair. X: list of feature values per candidate pair (X is our feature map)
  y.append(label)

print(columns)
for x in X:
  print(x)

print(" ")
print(y)

['left_distinct_ratio', 'right_distinct_ratio', 'jaccard_overlap', 'left_containment', 'right_containment', 'value_range_overlap', 'same_dtype', 'left_leftness', 'right_leftness', 'left_sorted', 'right_sorted', 'single_col', 'row_ratio']
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1, 0.0, 0.0, 1, 1, 1, 1.0]
[0.008771929824561403, 0.008771929824561403, 1.0, 1.0, 1.0, 0, 1, 0.8, 0.8333333333333334, 0, 0, 1, 1.0]
[0.005847953216374269, 0.008771929824561403, 0.0, 0.0, 0.0, 0, 0, 0.4, 0.16666666666666666, 0, 0, 1, 1.0]
[0.19005847953216373, 0.4473684210526316, 0.0845771144278607, 0.26153846153846155, 0.1111111111111111, 0.15532981528282985, 1, 0.6, 0.5, 0, 0, 1, 1.0]
[0.008771929824561403, 0.2953216374269006, 0.0, 0.0, 0.0, 0, 1, 0.8, 0.6666666666666666, 0, 0, 1, 1.0]
 
[1, 0, 0, 0, 0]


In [ ]:
# Prepare the data for the model
X = pd.DataFrame(X, columns=columns)
y = np.array(y)

# Train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = GradientBoostingClassifier() # n_estimators=100, random_state=42, verbose=1
clf.fit(X_train, y_train);

In [ ]:
# Evaluate the model
y_pred = clf.predict(X_test)
print("Feature names:", list(X.columns))
print("\nAccuracy:", clf.score(X_test, y_test))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Feature names: ['left_distinct_ratio', 'right_distinct_ratio', 'jaccard_overlap', 'left_containment', 'right_containment', 'value_range_overlap', 'same_dtype', 'left_leftness', 'right_leftness', 'left_sorted', 'right_sorted', 'single_col', 'row_ratio']

Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



### Groupby Features Extraction

In [ ]:
# dispplay titanic dataframe
print(titanic_df.shape)
titanic_df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# do a simple groupby operation
grouped_table = titanic_df.groupby(['Pclass', 'Sex'], observed=True)['Survived'].agg(['mean', 'count']).reset_index()
print(grouped_table.shape)
grouped_table

(6, 4)


,Pclass,Sex,mean,count
0,1,female,0.968085,94
1,1,male,0.368852,122
2,2,female,0.921053,76
3,2,male,0.157407,108
4,3,female,0.500000,144
5,3,male,0.135447,347


In [ ]:
# group joined_tables by sex and calculate the mean of fare
grouped_table = joined_tables.groupby(['class', 'sex'], observed=True)['survived'].agg(['mean', 'count']).reset_index()

# rename mean and count columns
grouped_table.columns = ['class', 'sex', 'survival_rate', 'num_passengers']

# display the first 5 rows
print(grouped_table.head())

In [ ]:
# Feature 1: Distinct-value-count

for col in survived_left.columns:
  distinct_values_count = survived_left[col].nunique()
  print(f"{col}")
  print(f"Distinct vaues count: {distinct_values_count}")
  distinct_values_ratio = distinct_values_count / len(survived_left)
  print(f"Distinct vaues ratio: {distinct_values_ratio}\n")

PassengerId
Distinct vaues count: 342
Distinct vaues ratio: 1.0

Name
Distinct vaues count: 342
Distinct vaues ratio: 1.0

Sex
Distinct vaues count: 2
Distinct vaues ratio: 0.005847953216374269

Age
Distinct vaues count: 65
Distinct vaues ratio: 0.19005847953216373

Embarked
Distinct vaues count: 3
Distinct vaues ratio: 0.008771929824561403



In [ ]:
# Feature 2: Column-data-type

for col in survived_left.columns:
  data_type = survived_left[col].dtype
  print(f"{col}: {data_type}")

print(" ")
for col in survived_right.columns:
  data_type = survived_right[col].dtype
  print(f"{col}: {data_type}")

print(" ")
for col in survived_left.columns:
  for col2 in survived_right.columns:
    if survived_left[col].dtype == survived_right[col2].dtype:
      print(f"{col} and {col2} have the same data type\n")

PassengerId: int64
Name: object
Sex: object
Age: float64
Embarked: object
 
PassengerId: int64
Pclass: int64
Ticket: object
Fare: float64
Cabin: object
Embarked: object
 
PassengerId and PassengerId have the same data type

PassengerId and Pclass have the same data type

Name and Ticket have the same data type

Name and Cabin have the same data type

Name and Embarked have the same data type

Sex and Ticket have the same data type

Sex and Cabin have the same data type

Sex and Embarked have the same data type

Age and Fare have the same data type

Embarked and Ticket have the same data type

Embarked and Cabin have the same data type

Embarked and Embarked have the same data type



In [ ]:
# Feature 3: Left-ness

for col in survived_left.columns:
  leftness = survived_left.columns.get_loc(col) / (survived_left.shape[1])
  print(f"{col}: {leftness}")

print(" ")
for col in survived_right.columns:
  leftness = survived_right.columns.get_loc(col) / (survived_right.shape[1])
  print(f"{col}: {leftness:.2}")

PassengerId: 0.0
Name: 0.2
Sex: 0.4
Age: 0.6
Embarked: 0.8
 
PassengerId: 0.0
Pclass: 0.17
Ticket: 0.33
Fare: 0.5
Cabin: 0.67
Embarked: 0.83


In [ ]:
# Feature 4: Emptiness (% of null values)
# A good Groupby candidate has low emptiness

for col in survived_left.columns:
  null_count = survived_left[col].isnull().sum()
  null_ratio = null_count / len(survived_left)
  print(f"{col}: {null_ratio}")

PassengerId: 0.0
Name: 0.0
Sex: 0.0
Age: 0.15204678362573099
Embarked: 0.005847953216374269


In [ ]:
# Feature 5: Value-range (for numeric columns)
# (same as above in Join Feature Extraction)

for col in survived_left.columns:
  for col2 in survived_right.columns:
    if pd.api.types.is_numeric_dtype(survived_left[col]) and pd.api.types.is_numeric_dtype(survived_right[col2]):
      range1 = (survived_left[col].min(), survived_left[col].max())
      range2 = (survived_right[col2].min(), survived_right[col2].max())
      inter = max(0, min(range1[1], range2[1]) - max(range1[0], range2[0]))
      union_range = max(range1[1], range2[1]) - min(range1[0], range2[0])
      value_range_overlap = inter / union_range if union_range > 0 else 0
      print(f"Value range overlap between {col} and {col2}: {value_range_overlap}")

Value range overlap between PassengerId and PassengerId: 1.0
Value range overlap between PassengerId and Pclass: 0.0011248593925759281
Value range overlap between PassengerId and Fare: 0.573403595505618
Value range overlap between Age and PassengerId: 0.08768182737921265
Value range overlap between Age and Pclass: 0.025131942699170646
Value range overlap between Age and Fare: 0.15532981528282985


In [ ]:
# Feature 6: Peak-frequency

for col in survived_left.columns:
  value_counts = survived_left[col].value_counts(dropna=True)
  #print(f"{col}: {value_counts}")
  print(col)
  peak_frequency = value_counts.iloc[0] # count of the most frequent value
  print(f"Peak frequency: {peak_frequency}")
  peak_frequency_ratio = peak_frequency / len(survived_left) # ratio of peak_count to total len of the column
  print(f"Peak frequency ratio: {peak_frequency_ratio}")
  print(" ")

PassengerId
Peak frequency: 1
Peak frequency ratio: 0.0029239766081871343
 
Name
Peak frequency: 1
Peak frequency ratio: 0.0029239766081871343
 
Sex
Peak frequency: 233
Peak frequency ratio: 0.6812865497076024
 
Age
Peak frequency: 15
Peak frequency ratio: 0.043859649122807015
 
Embarked
Peak frequency: 217
Peak frequency ratio: 0.6345029239766082
 


In [ ]:
# Feature 7: Column-name-related features

# define some custom common names
common_groupby_terms = ['name', 'id', 'category', 'type', 'group', 'class', 'gender', 'year', 'month', 'day', 'region', 'country', 'state', 'city', 'quarter', 'segment', 'sector']
common_agg_terms = ['amount', 'count', 'sum', 'revenue', 'profit', 'sales', 'quantity', 'price', 'total', 'average', 'score', 'value', 'rate', 'ratio', 'percentage']

groupby_term = False
aggregation_term = False

for col in survived_left.columns:
  col_name_lower = col.lower()
  #print(f"{col}: {col_name_lower}")
  print(f"{col}")
  if col_name_lower in common_groupby_terms:
    print("Common groupby term")
    groupby_term = True
  if col_name_lower in common_agg_terms:
    aggregation_term = True
    print("Common aggregation term")
  print(" ")

PassengerId
 
Name
Common groupby term
 
Sex
 
Age
 
Embarked
 


### Groupby Model Training and Evaluation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Manually label columns as GroupBy=1 or not=0
# For real data, these would come from notebook replay logs(param.json file)
column_labels = {
    'PassengerId': 1,  # ID
    'Name': 1,         # group-by on name is plausible
    'Sex': 1,          # categorical dimension
    'Age': 0,          # continuous measure
    'Embarked': 1      # category
}

In [ ]:
# Groupby Feature Extraction
def extract_groupby_features(df: pd.DataFrame, col: str) -> dict:
    features = {}

    # Feature 1: Distinct-value-count and ratio
    features['distinct_count'] = df[col].nunique()
    features['distinct_ratio'] = features['distinct_count'] / len(df)

    # Feature 2: Column-data-type
    features['is_string'] = int(pd.api.types.is_string_dtype(df[col]))
    features['is_numeric'] = int(pd.api.types.is_numeric_dtype(df[col]))

    # Feature 3: Left-ness (column position)
    features['leftness'] = df.columns.get_loc(col) / len(df.columns)

    # Feature 4: Emptiness (% of null values)
    null_count = df[col].isnull().sum()
    features['emptiness'] = null_count / len(df)

    # Feature 5: Peak frequency
    value_counts = df[col].value_counts(dropna=True)
    if not value_counts.empty:
        peak_count = value_counts.iloc[0]
        features['peak_freq'] = peak_count
        features['peak_ratio'] = peak_count / len(df)
    else:
        features['peak_freq'] = 0
        features['peak_ratio'] = 0

    # Feature 6: Column-name match
    common_groupby_terms = ['name', 'id', 'category', 'type', 'group', 'class', 'gender', 'year', 'month', 'day', 'region', 'country', 'state', 'city', 'quarter', 'segment', 'sector']
    common_agg_terms = ['amount', 'count', 'sum', 'revenue', 'profit', 'sales', 'quantity', 'price', 'total', 'average', 'score', 'value', 'rate', 'ratio', 'percentage']
    name_lower = col.lower()
    features['name_is_groupby_term'] = int(any(term in name_lower for term in common_groupby_terms))
    features['name_is_agg_term'] = int(any(term in name_lower for term in common_agg_terms))

    return features

In [ ]:
# Build Dataset (same process as in Join)
X = []
y = []
columns = list(column_labels.keys())

for col in columns:
    feats = extract_groupby_features(survived_left, col)
    X.append(list(feats.values()))
    y.append(column_labels[col])

feature_names = list(feats.keys())
X = pd.DataFrame(X, columns=feature_names)
y = np.array(y)

In [ ]:
# Train and Evaluate the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = GradientBoostingClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
#print(y_pred)

print("Feature names:", feature_names)
print("\nAccuracy:", clf.score(X_test, y_test))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Feature names: ['distinct_count', 'distinct_ratio', 'is_string', 'is_numeric', 'leftness', 'emptiness', 'peak_freq', 'peak_ratio', 'name_is_groupby_term', 'name_is_agg_term']

Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



### Pivot Features Extraction

In [ ]:
# display again the head of titanic dataframe
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# example of a simple pivot operation (count of survivors by Sex and Pclass)
passengers = titanic_df.pivot_table(
    index='Sex',             # rows (Index parameter)
    columns='Pclass',        # columns (Header parameter)
    values='Survived',       # the value we're aggregating (Aggregation Column parameter)
    aggfunc='sum',           # how to aggregate (Aggregation function parameter)
    observed=True
)

# display the first 5 rows
passengers.head()

Pclass,1,2,3
Sex,,,
female,91,70,72
male,45,17,47


In [ ]:
# Feature 1: Emptiness-Reduction-Ratio (ERR)

# GOAL: When building a pivot table, avoid NULLs by choosing compatible columns to group together on the same side (index or header).
#
# Case 1: Sector vs. Company (Sector as Index and Company as Header/Column)
#   - 20 unique Sectors
#   - 1000 unique Companies (each belongs to 1 Sector)
#   - Only 1000 valid combinations (Company → Sector is 1-to-1)
#   - Total possible combinations = 20 * 1000 = 20,000
#   - ERR = 20,000 / 1000 = 20 -> very sparse if split - too may NULL values (imagine a table 20x1000 with only one value per row!)
#
#   → High ERR ⇒ Don't split Sector & Company — put them on same side.
#
# Case 2: Sector vs. Year (Sector as Index and Year as Header/Column)
#   - 20 Sectors, 3 Years
#   - All combinations exist (each Sector has all Years) ⇒ 20 * 3 = 60
#   - ERR = 60 / 60 = 1 (imagine a table 20x3 with a value at every cell!)
#
#   → Low ERR ⇒ It's fine to split Sector & Year — few NULLs expected.

def compute_emptiness_reduction_ratio(df: pd.DataFrame, col1: str, col2: str) -> float:
  """ Computes Emptiness Reduction Ratio (ERR) between two categorical columns. """
  unique_1 = df[col1].dropna().nunique()
  unique_2 = df[col2].dropna().nunique()

  # Create pivot with counts
  pivot = df.pivot_table(index=col1, columns=col2, values='PassengerId', aggfunc='count')

  observed = pivot.notnull().sum().sum()

  if observed == 0:
    return float('inf')
  return (unique_1 * unique_2) / observed

# Example: use 'Sex' and 'Pclass' from Titanic
err_sex_pclass = compute_emptiness_reduction_ratio(titanic_df, 'Sex', 'Pclass')
print(f"ERR (Sex, Pclass): {err_sex_pclass:.2f}")

# Try another less related pair
err_sex_cabin = compute_emptiness_reduction_ratio(titanic_df, 'Sex', 'Cabin')
print(f"ERR (Sex, Cabin): {err_sex_cabin:.2f}")

ERR (Sex, Pclass): 1.00
ERR (Sex, Cabin): 1.72


In [ ]:
# Feature 2: Column-Position-Difference (columns close to each other are often related)

for col1 in survived_left.columns:
  for col2 in survived_right.columns:
    col1_position = survived_left.columns.get_loc(col1)
    col2_position = survived_right.columns.get_loc(col2)
    position_difference = abs(col1_position - col2_position)
    print(f"Column 1: {col1} | Column 2: {col2}")
    print(f"Position difference between {col1} and {col2}: {position_difference}\n")

Column 1: PassengerId | Column 2: PassengerId
Position difference between PassengerId and PassengerId: 0

Column 1: PassengerId | Column 2: Pclass
Position difference between PassengerId and Pclass: 1

Column 1: PassengerId | Column 2: Ticket
Position difference between PassengerId and Ticket: 2

Column 1: PassengerId | Column 2: Fare
Position difference between PassengerId and Fare: 3

Column 1: PassengerId | Column 2: Cabin
Position difference between PassengerId and Cabin: 4

Column 1: PassengerId | Column 2: Embarked
Position difference between PassengerId and Embarked: 5

Column 1: Name | Column 2: PassengerId
Position difference between Name and PassengerId: 1

Column 1: Name | Column 2: Pclass
Position difference between Name and Pclass: 0

Column 1: Name | Column 2: Ticket
Position difference between Name and Ticket: 1

Column 1: Name | Column 2: Fare
Position difference between Name and Fare: 2

Column 1: Name | Column 2: Cabin
Position difference between Name and Cabin: 3

Co

In [ ]:
# AMPT: Affinity-Maximizing Pivot Table Split (Similar to Section 4.3 in Auto-Suggest Paper)
#
# GOAL: Given a set of candidate dimension columns (used in GroupBy-like pivoting), decide which to place as Pivot index (rows) and which as header (columns), in order to avoid NULLs #
#       in the resulting pivot table. Uses Feature 1 (Emptiness-Reduction-Ratio) and Feature 2 (Column-Position-Difference).
#
# Steps:
# 1. Start with a list of candidate dimension columns (e.g., ['Sex', 'Pclass', 'Embarked']).
#
# 2. For each pair of columns, compute an affinity score that captures:
#    - Emptiness-Reduction-Ratio (ERR): measures how sparse the pivot would be if these two columns are split.
#        • Lower ERR → higher compatibility for splitting → columns are weakly related ⇒ safe to put on opposite sides
#        • Higher ERR → columns are strongly related ⇒ keep on the same side to avoid NULLs
#
#    - Column Position Difference: columns that appear closer together in the original table
#      are more likely to be semantically related — smaller position difference = higher affinity.
#
# 3. Build a weighted graph:
#    - Nodes = candidate dimension columns
#    - Edges = pairwise affinity scores
#
# 4. Apply a graph partitioning algorithm (e.g., Kernighan–Lin min-cut) to split the columns into two groups:
#    - One group becomes the Pivot Index (rows)
#    - The other becomes the Pivot Header (columns)
#
# 5. This partition maximizes intra-group affinity and minimizes inter-group sparsity, resulting in a more compact, less NULL-filled pivot table.


import pandas as pd
import itertools
import networkx as nx

# Compute affinity score between two columns
def compute_affinity(df: pd.DataFrame, col1: str, col2: str) -> float:
    # Feature 1: Inverse ERR (high affinity if low ERR)
    unique_1 = df[col1].dropna().nunique()
    unique_2 = df[col2].dropna().nunique()
    pivot = df.pivot_table(index=col1, columns=col2, values='PassengerId', aggfunc='count')
    observed = pivot.notnull().sum().sum()
    err = (unique_1 * unique_2) / observed if observed > 0 else float('inf')
    err_affinity = 1 / err if err != float('inf') else 0  # higher ERR means lower affinity score and vice verce

    # Feature 2: Column position closeness
    pos_diff = abs(df.columns.get_loc(col1) - df.columns.get_loc(col2))
    pos_affinity = 1 / (1 + pos_diff)  # closeness → higher score -> Example: col1 at position 1, col2 at position 2 → pos_diff = 1 → pos_affinity = 1 / (1 + 1) = 0.5
    #                                                                         col1 at position 1, col2 at position 4 → pos_diff = 3 → pos_affinity = 1 / (1 + 3) = 0.25

    # Combine (simple weighted average)
    return 0.7 * err_affinity + 0.3 * pos_affinity # for this, authors trained a regression model!

# Run AMPT to split columns into index and header
def run_ampt(df: pd.DataFrame, dim_cols: list):
    G = nx.Graph()

    # Add nodes
    for col in dim_cols:
        G.add_node(col)

    # Add weighted edges using affinity scores
    for col1, col2 in itertools.combinations(dim_cols, 2):
        affinity = compute_affinity(df, col1, col2)
        G.add_edge(col1, col2, weight=affinity)

    # Partition the graph into two groups using the Kernighan–Lin algorithm.
    # It tries to maximize intra-group edge weights (keep high-affinity columns together) and minimize the total weight of edges cut between the two groups (low-affinity separation).
    # Returns:
    #   - cut_value: total weight of edges crossing between the two groups (minimized)
    #   - group1: list of columns assigned to one side (e.g., Pivot index)
    #   - group2: list of columns assigned to the other side (e.g., Pivot header)
    cut_value, (group1, group2) = nx.algorithms.community.kernighan_lin_bisection(G, weight='weight')

    cut_value, (group1, group2) = nx.algorithms.community.kernighan_lin_bisection(G, weight='weight')

    return group1, group2

In [ ]:
# Choose dimension-like columns from Titanic => All of these are candidate dimensions from a GroupBy perspective
dim_cols = ['Sex', 'Pclass', 'Embarked']

index_group, header_group = run_ampt(titanic_df, dim_cols)

print("Recommended Index Columns:", index_group)
print("Recommended Header Columns:", header_group)

Recommended Index Columns: Pclass
Recommended Header Columns: Sex


In [ ]:
# Simple End-to-End Pivot Feature Extraction Pipeline (with Groupby block - not the official!)

# Step 1: GroupBy Feature Extraction
def extract_groupby_features(df: pd.DataFrame, col: str) -> dict:
    features = {}
    features['distinct_count'] = df[col].nunique()
    features['distinct_ratio'] = features['distinct_count'] / len(df)
    features['is_string'] = int(pd.api.types.is_string_dtype(df[col]))
    features['is_numeric'] = int(pd.api.types.is_numeric_dtype(df[col]))
    features['leftness'] = df.columns.get_loc(col) / len(df.columns)
    null_count = df[col].isnull().sum()
    features['emptiness'] = null_count / len(df)
    value_counts = df[col].value_counts(dropna=True)
    if not value_counts.empty:
        peak_count = value_counts.iloc[0]
        features['peak_freq'] = peak_count
        features['peak_ratio'] = peak_count / len(df)
    else:
        features['peak_freq'] = 0
        features['peak_ratio'] = 0
    common_groupby_terms = ['name', 'id', 'category', 'type', 'group', 'class', 'gender', 'year', 'month', 'day', 'region', 'country', 'state', 'city', 'quarter', 'segment', 'sector']
    common_agg_terms = ['amount', 'count', 'sum', 'revenue', 'profit', 'sales', 'quantity', 'price', 'total', 'average', 'score', 'value', 'rate', 'ratio', 'percentage']
    name_lower = col.lower()
    features['name_is_groupby_term'] = int(any(term in name_lower for term in common_groupby_terms))
    features['name_is_agg_term'] = int(any(term in name_lower for term in common_agg_terms))
    return features

# Step 2: Use a trained GroupBy predictor to select candidate columns
# (We'll simulate the classifier here with a dummy rule or pretrained one)
def select_dimension_candidates(df: pd.DataFrame, clf: GradientBoostingClassifier = None):
    features_list = []
    cols = []
    for col in df.columns:
        try:
            feats = extract_groupby_features(df, col)
            features_list.append(list(feats.values()))
            cols.append(col)
        except Exception:
            continue
    X = pd.DataFrame(features_list, columns=feats.keys())
    if clf:
        y_pred = clf.predict(X) # it is trained on  previous cell above!
        return [col for col, pred in zip(cols, y_pred) if pred == 1]
    else:
        # fallback: rule-based (simulate good dimension candidates heuristically)
        return [col for col, feats in zip(cols, features_list) if feats[1] < 0.2 and feats[5] < 0.1]  # low distinct_ratio and low emptiness


# Step 3: Use AMPT to split dimension columns into index vs. header
from itertools import combinations
import networkx as nx

def compute_affinity(df: pd.DataFrame, col1: str, col2: str) -> float:
    u1 = df[col1].dropna().nunique()
    u2 = df[col2].dropna().nunique()
    pivot = df.pivot_table(index=col1, columns=col2, values='PassengerId', aggfunc='count')
    observed = pivot.notnull().sum().sum()
    err = (u1 * u2) / observed if observed > 0 else float('inf')
    err_aff = 1 / err if err != float('inf') else 0
    pos_diff = abs(df.columns.get_loc(col1) - df.columns.get_loc(col2))
    pos_aff = 1 / (1 + pos_diff)
    return 0.7 * err_aff + 0.3 * pos_aff

def run_ampt(df: pd.DataFrame, dim_cols: list):
    G = nx.Graph()
    for col in dim_cols:
        G.add_node(col)
    for col1, col2 in combinations(dim_cols, 2):
        affinity = compute_affinity(df, col1, col2)
        G.add_edge(col1, col2, weight=affinity)
    (group1, group2) = nx.algorithms.community.kernighan_lin_bisection(G, weight='weight')
    return group1, group2

# Step 4: Run full pipeline on Titanic
df = pd.read_csv("titanic.csv")

dim_candidates = select_dimension_candidates(df)

if len(dim_candidates) >= 2:
    index_cols, header_cols = run_ampt(df, dim_candidates)
    print("GroupBy-based candidate dimensions:", dim_candidates)
    print("Recommended Pivot Index:", index_cols)
    print("Recommended Pivot Header:", header_cols)
else:
    print("Not enough dimension candidates found.")

GroupBy-based candidate dimensions: ['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
Recommended Pivot Index: {'Pclass', 'Parch', 'Survived'}
Recommended Pivot Header: {'SibSp', 'Sex', 'Embarked'}


### Pivot Model Training and Evaluation

In [ ]:
# Pipeline: Pivot Column Prediction using GroupBy Features + AMPT (Similar to Auto-Suggest, not exactly the same!)
#
# Step 1: Extract GroupBy features for each column in the table.
# Step 2: Use a GroupBy classifier (simulated here) to detect dimension columns.
# Step 3: For each pair of dimension columns, compute pivot-specific features.
# Step 4: Train a regression model to learn affinity scores (instead of hardcoded 0.7/0.3 weights).
# Step 5: Use AMPT with the learned model to split dimensions into index vs header.

import pandas as pd
import numpy as np
import networkx as nx
from itertools import combinations
from sklearn.linear_model import LinearRegression

df = pd.read_csv("titanic.csv")

# ----- Step 1: GroupBy Feature Extraction -----
def extract_groupby_features(df: pd.DataFrame, col: str) -> dict:
    features = {}
    features['distinct_count'] = df[col].nunique()
    features['distinct_ratio'] = features['distinct_count'] / len(df)
    features['is_string'] = int(pd.api.types.is_string_dtype(df[col]))
    features['is_numeric'] = int(pd.api.types.is_numeric_dtype(df[col]))
    features['leftness'] = df.columns.get_loc(col) / len(df.columns)
    features['emptiness'] = df[col].isnull().sum() / len(df)
    value_counts = df[col].value_counts(dropna=True)
    peak = value_counts.iloc[0] if not value_counts.empty else 0
    features['peak_ratio'] = peak / len(df)
    name = col.lower()
    features['name_is_groupby_term'] = int(any(term in name for term in ['sex', 'class', 'name', 'id', 'type']))
    features['name_is_agg_term'] = int(any(term in name for term in ['fare', 'age', 'score', 'value']))
    return features

# ----- Step 2: Simulate GroupBy classifier -----
# In the Auto-Suggest paper (Section 4.2), this step is performed using a trained Gradient Boosting Tree (GBT) classifier, which learns from features like distinct ratio,
# data type, emptiness, peak frequency, and name indicators. Here, we approximate that logic using a simple rule-based heuristic for illustration.
def select_dimension_candidates(df):
    candidates = []
    for col in df.columns:
        feats = extract_groupby_features(df, col)
        if feats['distinct_ratio'] < 0.8 and feats['emptiness'] < 0.8 and feats['is_string']:
            candidates.append(col)
    return candidates

# ----- Step 3: Extract features between column pairs -----
def extract_pairwise_features(df, col1, col2):
    u1, u2 = df[col1].dropna().nunique(), df[col2].dropna().nunique()
    pivot = df.pivot_table(index=col1, columns=col2, values='PassengerId', aggfunc='count')
    observed = pivot.notnull().sum().sum()
    err = (u1 * u2) / observed if observed > 0 else float('inf')
    err_aff = 1 / err if err != float('inf') else 0
    pos_diff = abs(df.columns.get_loc(col1) - df.columns.get_loc(col2))
    pos_aff = 1 / (1 + pos_diff)
    return [err_aff, pos_aff]

# ----- Step 4: Train regression model on dummy labels -----
# Instead of using fixed weights like: affinity = 0.7 * err_aff + 0.3 * pos_aff, we train a regression model to learn the optimal combination of these features
# (ERR and column position) based on labeled examples of column-pair relationships. This follows the paper's approach in Section 4.3, where affinity scores are learned
# from data to guide the pivot column split more accurately. We train on our intila dataset for Pivot operation and NOT on the data coming from groupby model's output !!

# Format: (col1, col2, label: 1 = keep together, 0 = split OK)
training_data = [
    ('Sex', 'Embarked', 1),
    ('Sex', 'Pclass', 0),
    ('Embarked', 'Pclass', 0)
]

X_train = []
y_train = []
for col1, col2, label in training_data:
    X_train.append(extract_pairwise_features(df, col1, col2))
    y_train.append(label)

reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

# ----- Step 5: Run AMPT using learned model -----
def compute_affinity(df: pd.DataFrame, col1: str, col2: str, model) -> float:
    x = np.array(extract_pairwise_features(df, col1, col2)).reshape(1, -1)
    affinity_score = model.predict(x)[0]
    return affinity_score

def run_ampt(df: pd.DataFrame, dim_cols: list, model):
    G = nx.Graph()
    for col in dim_cols:
        G.add_node(col)
    for col1, col2 in combinations(dim_cols, 2):
        affinity = compute_affinity(df, col1, col2, model)
        G.add_edge(col1, col2, weight=affinity)
    group1, group2 = nx.algorithms.community.kernighan_lin_bisection(G, weight='weight')
    return group1, group2

# ----- Execute Full Pipeline -----
dim_candidates = select_dimension_candidates(df)

if len(dim_candidates) >= 2:
    index_cols, header_cols = run_ampt(df, dim_candidates, reg_model)
    print("Step 2 → Dimension candidates:", dim_candidates)
    print("Step 5 → Recommended Pivot Index:", index_cols)
    print("Step 5 → Recommended Pivot Header:", header_cols)
else:
    print("Not enough dimension candidates found.")

Step 2 → Dimension candidates: ['Sex', 'Ticket']
Step 5 → Recommended Pivot Index: {'Sex'}
Step 5 → Recommended Pivot Header: {'Ticket'}


### Unpivot Features Extraction

In [ ]:
# Load data (again!)
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# Melt Age and Fare into long format
unpivoted = pd.melt(
    df,
    id_vars=['PassengerId', 'Name'],         # columns to keep
    value_vars=['Age', 'Fare'],              # columns to unpivot
    var_name='Measure',                      # new column for 'Age' or 'Fare'
    value_name='Value'                       # new column for numeric values
)

# Unpivot (melt) the 'Age' and 'Fare' columns into long format. Each row will represent a single measurement per passenger.
# We expect to see two rows per passenger: one for 'Age' and one for 'Fare', with the column name in 'Measure' and the corresponding value in 'Value'.

# display the top of unpivoted tabel
unpivoted.head()

,PassengerId,Name,Measure,Value
0,1,"Braund, Mr. Owen Harris",Age,22.0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Age,38.0
2,3,"Heikkinen, Miss. Laina",Age,26.0
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Age,35.0
4,5,"Allen, Mr. William Henry",Age,35.0


In [ ]:
# display the tail of unpivoted tabel
unpivoted.tail()

,PassengerId,Name,Measure,Value
1777,887,"Montvila, Rev. Juozas",Fare,13.00
1778,888,"Graham, Miss. Margaret Edith",Fare,30.00
1779,889,"Johnston, Miss. Catherine Helen ""Carrie""",Fare,23.45
1780,890,"Behr, Mr. Karl Howell",Fare,30.00
1781,891,"Dooley, Mr. Patrick",Fare,7.75


In [ ]:
# Unpivot Feature Extraction (or same as Groupby from above)
def extract_unpivot_features(df: pd.DataFrame, col: str) -> dict:
    features = {}

    # Feature 1: Data type (is it numeric?)
    features['is_numeric'] = int(pd.api.types.is_numeric_dtype(df[col]))

    # Feature 2: Distinct-value ratio
    distinct = df[col].nunique()
    features['distinct_ratio'] = distinct / len(df)

    # Feature 3: Emptiness
    features['null_ratio'] = df[col].isnull().sum() / len(df)

    # Feature 4: Peak frequency ratio
    value_counts = df[col].value_counts(dropna=True)
    peak = value_counts.iloc[0] if not value_counts.empty else 0
    features['peak_ratio'] = peak / len(df)

    # Feature 5: Name signal
    name = col.lower()
    features['name_is_metric'] = int(any(term in name for term in ['score', 'value', 'rate', 'amount', 'total', 'fare', 'age']))

    return features

In [ ]:
# Candidate column list from Titanic (we'll manually label them in the next section)
candidate_columns = ['Age', 'Fare', 'SibSp', 'Parch', 'Sex', 'Embarked']

# Extract features for all candidate columns
unpivot_features = []
unpivot_labels = []  # will fill in next section
columns_used = []

for col in candidate_columns:
    try:
        feats = extract_unpivot_features(df, col)
        unpivot_features.append(feats)
        columns_used.append(col)
    except Exception as e:
        print(f"Skipping column {col}: {e}")

# Convert to DataFrame (ready for training/evaluation in the next section)
import pandas as pd

X_unpivot = pd.DataFrame(unpivot_features)
print("Unpivot feature matrix:\n", X_unpivot)
print("Candidate columns extracted:", columns_used)

Unpivot feature matrix:
    is_numeric  distinct_ratio  null_ratio  peak_ratio  name_is_metric
0           1        0.098765    0.198653    0.033670               1
1           1        0.278339    0.000000    0.048260               1
2           1        0.007856    0.000000    0.682379               0
3           1        0.007856    0.000000    0.760943               0
4           0        0.002245    0.000000    0.647587               0
5           0        0.003367    0.002245    0.722783               0
Candidate columns extracted: ['Age', 'Fare', 'SibSp', 'Parch', 'Sex', 'Embarked']


In [ ]:
# Compute compatibility scores for column pairs using trained Pivot regression model

# Feature 1: Inverse Emptiness Reduction Ratio (1/ERR)
# Feature 2: Column position difference

from itertools import combinations
import numpy as np

def extract_pairwise_unpivot_features(df, col1, col2):
    # ERR if melted into pivot
    u1 = df[col1].dropna().nunique()
    u2 = df[col2].dropna().nunique()
    pivot = df.pivot_table(index=col1, columns=col2, values='PassengerId', aggfunc='count')
    observed = pivot.notnull().sum().sum()
    err = (u1 * u2) / observed if observed > 0 else float('inf')
    err_score = 1 / err if err != float('inf') else 0

    # Column position difference
    pos_diff = abs(df.columns.get_loc(col1) - df.columns.get_loc(col2))
    pos_score = 1 / (1 + pos_diff)

    return [err_score, pos_score]

# Generate pairwise compatibility scores
def compute_compatibility_matrix(df, columns, clf):
    compat_scores = {}
    for col1, col2 in combinations(columns, 2):
        features = extract_pairwise_unpivot_features(df, col1, col2)
        score = reg_model.predict([[features[0], features[1]]])[0]
        compat_scores[(col1, col2)] = score
    return compat_scores

In [ ]:
# Test script: Compute compatibility matrix using trained clf

# Pick a set of candidate measure columns from Titanic
test_columns = ['Age', 'Fare', 'SibSp', 'Parch']

# Run compatibility scoring
compatibility_scores = compute_compatibility_matrix(df, test_columns, reg_model)  # reg_model is the trained model from above

# Print results
print("Pairwise compatibility scores:")
for (col1, col2), score in compatibility_scores.items():
    print(f"({col1}, {col2}): {score:.4f}")

Pairwise compatibility scores:
(Age, Fare): 0.3075
(Age, SibSp): -0.2511
(Age, Parch): 0.0592
(Fare, SibSp): 0.2144
(Fare, Parch): 0.0592
(SibSp, Parch): -0.2511


In [ ]:
# CMUT: Compatibility-Maximizing Unpivot Table (Exact Subset Search)
#
# GOAL: Given a list of predicted measure columns, find the optimal subset (M') to unpivot (melt together) by maximizing the average pairwise compatibility.
#
# Objective: maximize   (1 / |M'|²) * sum_{(ci, cj) ∈ M'} compat(ci, cj)
#
# This implementation performs **exhaustive search** over all subsets of columns (size ≥ 2) to find the one that maximizes this objective. It is exact, but:
# Computational complexity is exponential in the number of columns (NP-hard subset search), so this is only feasible for small input sets (≤ 10).
#
# In our research prototype (e.g., for the Titanic example), we can afford this. In a real production system, this would be replaced by a more scalable approach
# such as greedy clustering or threshold-based grouping.

def cmut(df, columns, clf):
    from itertools import combinations

    best_group = []
    best_avg_score = -1

    # Precompute all pairwise compatibilities (build compatibility graph)
    pairwise_scores = {}
    for col1, col2 in combinations(columns, 2):
        features = extract_pairwise_unpivot_features(df, col1, col2)
        score = reg_model.predict([[features[0], features[1]]])[0]
        #print(score)
        pairwise_scores[(col1, col2)] = score

    # Loop over all possible subsets (size ≥ 2) of the input columns
    for r in range(2, len(columns) + 1):
        # For each subset
        for subset in combinations(columns, r):
            total_score = 0
            count = 0

            # For each pair within the subset, sum up the pairwise compatibility scores
            for col1, col2 in combinations(subset, 2):
                # Get compatibility score between col1 and col2
                # Try both (col1, col2) and (col2, col1) in case the dict is asymmetric
                score = pairwise_scores.get((col1, col2)) or pairwise_scores.get((col2, col1)) or 0
                total_score += score
                count += 1 # count the number of pairs

            # Skip empty group
            if count == 0:
                continue

            # Calculate the average score across this group
            avg_score = total_score / (r * r)  # CMUT objective from the paper

            # Keep track of the best-scoring group so far
            if avg_score > best_avg_score:
                best_avg_score = avg_score
                best_group = list(subset)

    return best_group, best_avg_score

In [ ]:
# Test script: Run CMUT to find best group of columns to unpivot

# Reuse same test columns
test_columns = ['Age', 'Fare', 'SibSp', 'Parch']

# Run CMUT
best_group, avg_score = cmut(df, test_columns, reg_model)

# Show result
print("Best group to unpivot:", best_group)
print("Average compatibility score:", round(avg_score, 4))


Best group to unpivot: ['Age', 'Fare']
Average compatibility score: 0.0769


### Unpivot Model Training and Evaluation

In [ ]:
# # End-to-End CMUT Pipeline (CMUT: Compatibility-Maximizing Unpivot Table, Similar to Auto-Suggest, Section 4.4)
#
# GOAL: Automatically identify and group measure columns that should be unpivoted (using melt) into a long-form table structure.
#
# Process:
# 1. Extract per-column features (e.g., distinct ratio, null ratio, data type), similar to GroupBy column prediction.
#
# 2. Use a trained GroupBy-style classifier to predict which columns are unpivotable (columns that act as measures and are candidates for melting).
#
# 3. For each pair of predicted measure columns, extract pairwise features (Unpivot dataset):
#    - Feature 1: 1/ERR (inverse Emptiness Reduction Ratio if split via pivot)
#    - Feature 2: Column position difference
#    These are the same features used in the trained Pivot regression model.
#
# 4. Create the Unpivot feature space by applying the trained regression model
#    (from Pivot affinity training) to predict compatibility scores between column pairs.
#
# 5. Apply CMUT:
#    CMUT is solved by selecting the subset of measure columns M' that maximizes the **average pairwise compatibility score**, not just the total.
#
#    Objective: avg_compat(M') = (1 / |M'|²) * sum_{(ci, cj) ∈ M'} compat(ci, cj)
#
#    - For small column sets (e.g., ≤ 10), we can exhaustively evaluate all subsets.
#    - This ensures that only tightly related (mutually compatible) columns are grouped.
#
# Alternative (for large sets):
#    If the number of columns is large, a threshold-based approximation can be used instead:
#    - Initialize group with the highest scoring pair
#    - Add other columns whose compatibility with all existing group members exceeds a fixed threshold (e.g., 0.6)

# ------------------------------------------------------------------------------
# Full CMUT Pipeline using:
# - GroupBy classifier (clf) to identify unpivotable measure columns
# - Pivot-trained regression model (reg_model) to compute compatibility
# - CMUT to find optimal subset to unpivot
# ------------------------------------------------------------------------------
from itertools import combinations
import numpy as np

# --- Step 1: Extract per-column features (same as used in GroupBy prediction) ---
def extract_unpivot_features(df: pd.DataFrame, col: str) -> dict:
    features = {}
    features['is_numeric'] = int(pd.api.types.is_numeric_dtype(df[col]))
    features['distinct_ratio'] = df[col].nunique() / len(df)
    features['null_ratio'] = df[col].isnull().sum() / len(df)
    value_counts = df[col].value_counts(dropna=True)
    peak = value_counts.iloc[0] if not value_counts.empty else 0
    features['peak_ratio'] = peak / len(df)
    name = col.lower()
    features['name_is_metric'] = int(any(term in name for term in [
        'score', 'value', 'rate', 'amount', 'total', 'fare', 'age', 'price', 'cost', 'income'
    ]))
    return features

# --- Step 2: Predict measure columns using pretrained GroupBy classifier (clf) ---
def select_measure_columns(df, clf):
    X = []
    cols = []
    for col in df.columns:
        try:
            feats = extract_unpivot_features(df, col)
            X.append(list(feats.values()))
            cols.append(col)
        except Exception:
            continue
    import pandas as pd
    X_df = pd.DataFrame(X, columns=feats.keys())
    y_pred = clf.predict(X_df)
    return [col for col, pred in zip(cols, y_pred) if pred == 1]

# --- Step 3: Extract pairwise unpivot features ---
def extract_pairwise_unpivot_features(df, col1, col2):
    u1 = df[col1].dropna().nunique()
    u2 = df[col2].dropna().nunique()
    pivot = df.pivot_table(index=col1, columns=col2, values='PassengerId', aggfunc='count')
    observed = pivot.notnull().sum().sum()
    err = (u1 * u2) / observed if observed > 0 else float('inf')
    err_score = 1 / err if err != float('inf') else 0
    pos_diff = abs(df.columns.get_loc(col1) - df.columns.get_loc(col2))
    pos_score = 1 / (1 + pos_diff)
    return [err_score, pos_score]

# --- Step 4: Run CMUT using average compatibility objective (paper definition) ---
def cmut(df, columns, reg_model):
    best_group = []
    best_avg_score = -1
    pairwise_scores = {}

    for col1, col2 in combinations(columns, 2):
        features = extract_pairwise_unpivot_features(df, col1, col2)
        score = reg_model.predict([features])[0]
        pairwise_scores[(col1, col2)] = score

    for r in range(2, len(columns) + 1):
        for subset in combinations(columns, r):
            total_score = 0
            for col1, col2 in combinations(subset, 2):
                score = pairwise_scores.get((col1, col2)) or pairwise_scores.get((col2, col1)) or 0
                total_score += score
            avg_score = total_score / (r * r)
            if avg_score > best_avg_score:
                best_avg_score = avg_score
                best_group = list(subset)

    return best_group, best_avg_score

In [ ]:
from itertools import combinations

def cmut_greedy_threshold(df, columns, reg_model, threshold=0.6):
    """
    Greedy CMUT: Group columns to unpivot based on pairwise compatibility threshold.

    Args:
        df: DataFrame containing the data.
        columns: List of measure columns to consider.
        reg_model: Trained regression model (from Pivot) to score column pairs.
        threshold: Minimum compatibility score required to group columns.

    Returns:
        A list of melt groups (each group is a list of compatible columns).
    """

    # Step 1: Compute pairwise compatibility scores for all column pairs
    pairwise_scores = {}
    for col1, col2 in combinations(columns, 2):
        features = extract_pairwise_unpivot_features(df, col1, col2)
        score = reg_model.predict([features])[0]
        pairwise_scores[(col1, col2)] = score

    used = set()
    groups = []

    # Step 2: Greedily form groups
    for col in columns:
        if col in used:
            continue
        group = [col]
        for other in columns:
            if other == col or other in used:
                continue

            # Check if 'other' is compatible with all columns in the current group
            compatible = all(
                pairwise_scores.get((c, other), pairwise_scores.get((other, c), 0)) >= threshold
                for c in group
            )
            if compatible:
                group.append(other)
                used.add(other)

        used.update(group)
        groups.append(group)

    return groups

In [ ]:
measure_candidates = ['Age', 'Fare', 'SibSp', 'Parch']

# Exact CMUT
group, score = cmut(df, measure_candidates, reg_model)
print("Best Unpivot Group:", group)
print("Avg compatibility score:", round(score, 4))

# Threshold-based fallback
groups = cmut_greedy_threshold(df, measure_candidates, reg_model, threshold=0.8)
print("Greedy Unpivot Groups:", groups)

Best Unpivot Group: ['Age', 'Fare']
Avg compatibility score: 0.0769
Greedy Unpivot Groups: [['Age'], ['Fare'], ['SibSp'], ['Parch']]


## Predict Next Operator

### N-gram and RNN Models

In [ ]:
# Predict Next Operator (Auto-Suggest Paper)
#
# GOAL: Predict the next operator a user is likely to apply based on prior operator usage in a data preparation pipeline.
# We will simulate small synthetic sequences using real pandas operations.
# We train and evaluate next-operator prediction models using n-gram and rnn.

# dummy operator sequences
synthetic_sequences = [
    ['dropna', 'fillna', 'groupby', 'pivot'],
    ['dropna', 'groupby'],
    ['fillna', 'groupby', 'melt'],
    ['fillna', 'groupby', 'apply'],
    ['dropna', 'groupby', 'pivot', 'melt'],
    ['fillna', 'pivot', 'melt'],
    ['dropna', 'merge'],
    ['dropna', 'fillna', 'merge', 'groupby'],
    ['fillna', 'jsonnorm'],  # jsonnorm simulated
    ['groupby', 'pivot', 'apply']
]

In [ ]:
# Build training dataset: (X = prior ops), (y = next op)
train_X = []
train_y = []

# Generate training pairs (history -> next operation)
for seq in synthetic_sequences:
  for i in range(1, len(seq)):
    context = seq[:i]
    target = seq[i]
    train_X.append(context)
    train_y.append(target)

print("Training samples:")
for x, y in zip(train_X, train_y):
  print(f"History: {x} → Next: {y}")

Training samples:
History: ['dropna'] → Next: fillna
History: ['dropna', 'fillna'] → Next: groupby
History: ['dropna', 'fillna', 'groupby'] → Next: pivot
History: ['dropna'] → Next: groupby
History: ['fillna'] → Next: groupby
History: ['fillna', 'groupby'] → Next: melt
History: ['fillna'] → Next: groupby
History: ['fillna', 'groupby'] → Next: apply
History: ['dropna'] → Next: groupby
History: ['dropna', 'groupby'] → Next: pivot
History: ['dropna', 'groupby', 'pivot'] → Next: melt
History: ['fillna'] → Next: pivot
History: ['fillna', 'pivot'] → Next: melt
History: ['dropna'] → Next: merge
History: ['dropna'] → Next: fillna
History: ['dropna', 'fillna'] → Next: merge
History: ['dropna', 'fillna', 'merge'] → Next: groupby
History: ['fillna'] → Next: jsonnorm
History: ['groupby'] → Next: pivot
History: ['groupby', 'pivot'] → Next: apply


In [ ]:
# Encode operator sequences as integers using LabelEncoder.
#
# Why?
# - Traditional ML models (like logistic regression or n-gram tables) can't operate on strings.
# - So we convert operator names (e.g., 'groupby', 'pivot') to integer IDs.
#
# Note:
# - This is only suitable for simple models.
# - For RNNs or deep learning, we will use embedding layers instead of integer encoding.

from sklearn.preprocessing import LabelEncoder
import numpy as np

# Use the last operator in each history sequence as feature (unigram model)
X_last = [x[-1] for x in train_X]

# Combine all operator tokens to ensure LabelEncoder sees all possible values
all_ops = list(set(op for seq in train_X for op in seq) | set(train_y))
print("All operators:", all_ops)

le_op = LabelEncoder()
le_op.fit(all_ops)

# Encode features and targets
X_encoded = le_op.transform(X_last)
y_encoded = le_op.transform(train_y)

# Show label mapping
print("Operator label mapping:", dict(zip(le_op.classes_, le_op.transform(le_op.classes_))))

All operators: ['groupby', 'fillna', 'dropna', 'melt', 'merge', 'jsonnorm', 'apply', 'pivot']
Operator label mapping: {np.str_('apply'): np.int64(0), np.str_('dropna'): np.int64(1), np.str_('fillna'): np.int64(2), np.str_('groupby'): np.int64(3), np.str_('jsonnorm'): np.int64(4), np.str_('melt'): np.int64(5), np.str_('merge'): np.int64(6), np.str_('pivot'): np.int64(7)}


In [ ]:
# Build n-gram model (unigram → next op frequency)
from collections import defaultdict, Counter

# Build a 1-gram (bigram) model: map last operator → next operator counts
ngram_counts = defaultdict(Counter)

for x, y in zip(train_X, train_y):
  prev_op = x[-1]  # last operator in the history (context)
  ngram_counts[prev_op][y] += 1  # count transition: prev_op → y (next op)

print("1-gram model:")
for op, counter in ngram_counts.items():
  print(f"{op} → {counter}")

# Print normalized probabilities for each operator's transitions
print("\n1-gram transition probabilities:")
for op, counter in ngram_counts.items():
  total = sum(counter.values())
  probs = {k: round(v / total, 2) for k, v in counter.items()}
  print(f"{op} → {probs}")

1-gram model:
dropna → Counter({'fillna': 2, 'groupby': 2, 'merge': 1})
fillna → Counter({'groupby': 3, 'pivot': 1, 'merge': 1, 'jsonnorm': 1})
groupby → Counter({'pivot': 3, 'melt': 1, 'apply': 1})
pivot → Counter({'melt': 2, 'apply': 1})
merge → Counter({'groupby': 1})

1-gram transition probabilities:
dropna → {'fillna': 0.4, 'groupby': 0.4, 'merge': 0.2}
fillna → {'groupby': 0.5, 'pivot': 0.17, 'merge': 0.17, 'jsonnorm': 0.17}
groupby → {'pivot': 0.6, 'melt': 0.2, 'apply': 0.2}
pivot → {'melt': 0.67, 'apply': 0.33}
merge → {'groupby': 1.0}


In [ ]:
# Build a 2-gram model: (op1, op2) → next_op
ngram2_counts = defaultdict(Counter)

for x, y in zip(train_X, train_y):
    if len(x) < 2:
        continue  # skip if not enough history
    context = tuple(x[-2:])  # take last 2 ops
    ngram2_counts[context][y] += 1

# Display 2-gram transition probabilities
print("\n2-gram transition probabilities:")
for ctx, counter in ngram2_counts.items():
    total = sum(counter.values())
    probs = {k: round(v / total, 2) for k, v in counter.items()}
    print(f"{ctx} → {probs}")


2-gram transition probabilities:
('dropna', 'fillna') → {'groupby': 0.5, 'merge': 0.5}
('fillna', 'groupby') → {'pivot': 0.33, 'melt': 0.33, 'apply': 0.33}
('dropna', 'groupby') → {'pivot': 1.0}
('groupby', 'pivot') → {'melt': 0.5, 'apply': 0.5}
('fillna', 'pivot') → {'melt': 1.0}
('fillna', 'merge') → {'groupby': 1.0}


In [ ]:
# RNN-Based Operator Prediction Model
#
# GOAL: Predict the next operator based on a full sequence of previous operators.
#       This is a many-to-one sequence prediction task.
#
# We use an LSTM model trained on synthetic operator sequences.
# Each sequence is encoded as a list of operator token IDs using LabelEncoder.
#
# Workflow:
#   - Encode each operator as an integer using LabelEncoder
#   - Pad sequences to equal length
#   - Train an LSTM to predict the next operator

In [ ]:
# Step 1: Encode operator tokens as integers to prepare input for the RNN.
# - RNNs (LSTM in our case) expect integer sequences or embedded vectors as input.
# - Each operator (e.g., 'dropna', 'groupby', 'pivot') is treated like a word/token.
# - We use LabelEncoder to assign a unique integer ID to each operator.

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

# Encode operator vocabulary
all_ops = sorted(set(op for seq in synthetic_sequences for op in seq))
le_rnn = LabelEncoder()
le_rnn.fit(all_ops)
vocab_size = len(le_rnn.classes_)  # number of unique operators
print("Operator label mapping:", dict(zip(le_rnn.classes_, le_rnn.transform(le_rnn.classes_))))

Operator label mapping: {np.str_('apply'): np.int64(0), np.str_('dropna'): np.int64(1), np.str_('fillna'): np.int64(2), np.str_('groupby'): np.int64(3), np.str_('jsonnorm'): np.int64(4), np.str_('melt'): np.int64(5), np.str_('merge'): np.int64(6), np.str_('pivot'): np.int64(7)}


In [ ]:
# Step 2: Build training pairs (X = full history, y = next op)
X_seq = []
y_seq = []

for seq in synthetic_sequences:
  encoded = le_rnn.transform(seq)
  for i in range(1, len(encoded)):
    X_seq.append(encoded[:i])      # history
    y_seq.append(encoded[i])       # next op

print(X_seq)
print(y_seq)

# Step 3: Pad sequences
max_len = max(len(seq) for seq in X_seq)
X_pad = pad_sequences(X_seq, maxlen=max_len, padding='pre')
#print(X_pad)

# Step 4: One-hot encode targets
y_cat = to_categorical(y_seq, num_classes=vocab_size)

print(f"\nTotal training samples: {len(X_pad)}")
print(f"Vocabulary size: {vocab_size}")
print(f"Max sequence length: {max_len}")

[array([1]), array([1, 2]), array([1, 2, 3]), array([1]), array([2]), array([2, 3]), array([2]), array([2, 3]), array([1]), array([1, 3]), array([1, 3, 7]), array([2]), array([2, 7]), array([1]), array([1]), array([1, 2]), array([1, 2, 6]), array([2]), array([3]), array([3, 7])]
[np.int64(2), np.int64(3), np.int64(7), np.int64(3), np.int64(3), np.int64(5), np.int64(3), np.int64(0), np.int64(3), np.int64(7), np.int64(5), np.int64(7), np.int64(5), np.int64(6), np.int64(2), np.int64(6), np.int64(3), np.int64(4), np.int64(7), np.int64(0)]

Total training samples: 20
Vocabulary size: 8
Max sequence length: 3


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=16, input_length=max_len)) # Transform each operator token into a dense embedding vector (Embeddings capture semantic similarity — operators with similar usage contexts, e.g. 'groupby' and 'pivot' will learn similar vector representations)
model.add(LSTM(64))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(X_pad, y_cat, epochs=30, verbose=1)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.2500 - loss: 2.0786
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.2500 - loss: 2.0757
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.3000 - loss: 2.0729
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.4500 - loss: 2.0700
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.4500 - loss: 2.0670
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.4000 - loss: 2.0640
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.4000 - loss: 2.0610
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.4000 - loss: 2.0578
Epoch 9/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.3500 - loss: 2.0544
Epoch 10/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.3500 - loss: 2.0510
Epoch 11/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.3500 - loss: 2.0474
Epoch 12/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.3500 - loss: 2.0436
E

In [ ]:
# Predict the next operator given a list of previous ops
def predict_next_operator(seq_tokens):

  # To make prediction, we need to bring the input sequence to the form our model is trained on (encode+padding)
  encoded = le_rnn.transform(seq_tokens)
  padded = pad_sequences([encoded], maxlen=max_len, padding='pre')

  # Predict
  pred = model.predict(padded, verbose=0)
  pred_id = np.argmax(pred)
  return le_rnn.inverse_transform([pred_id])[0]

# Example prediction
test_seq = ['dropna', 'groupby']
predicted = predict_next_operator(test_seq)
print(f"Given history {test_seq} → Predicted next operator: {predicted}")

Given history ['dropna', 'groupby'] → Predicted next operator: groupby


### Combining Operation Sequence (RNN/n-gram) with Operator Features Simple


In [ ]:
# Final Next-Operator Prediction Model (MLP)
#
# We now combine:
# - Sequence context from the RNN or n-gram model (as softmax probs over ops)
# - Current table features (e.g., row count, nulls, data type counts)
#
# This simulates the full model described in the Auto-Suggest paper, where operator prediction depends not only on prior ops, but also on the data.
#
# Input = [RNN output (vector of op probs) + table features]
# Output = next operator (softmax over operator labels)

In [ ]:
# Extract simplified table-level features from the current DataFrame (Titanic toy version)
def extract_table_features(df):
    return np.array([
        len(df),                                # number of rows
        df.shape[1],                            # number of columns
        df.isnull().sum().sum(),                # total nulls
        df.select_dtypes(include='number').shape[1],  # numeric columns
        df.select_dtypes(include='object').shape[1]   # string/categorical columns
    ])

In [ ]:
# For each example, to produce the final training dataset (X, y), we'll get:
# - X_seq: RNN-predicted probabilities from history
# - X_tab: features from a synthetic "current table"
# - y: next operator

X_final = []
y_final = []

for x_seq, y_op in zip(train_X, train_y):
  # Sequence-based feature (RNN softmax prediction)
  try:
    rnn_input = le_rnn.transform(x_seq)
    padded = pad_sequences([rnn_input], maxlen=max_len, padding='pre')
    rnn_probs = model.predict(padded, verbose=0)[0]  # shape: (vocab_size,)
  except:
    continue

  print(f"\nSequence: {x_seq} → RNN probs: {rnn_probs}")

  # Table features (we simulate current table as Titanic slice)
  table_feats = extract_table_features(df.head(50))  # simulate a partial table

  print(f"Table features: {table_feats}")

  # Combine both into one vector
  combined_input = np.concatenate([rnn_probs, table_feats])
  X_final.append(combined_input)

  print(f"Combined input: {combined_input}")

  # Label: next op
  y_final.append(le_rnn.transform([y_op])[0])
  print(f"Label: {y_op}")


Sequence: ['dropna'] → RNN probs: [0.09382451 0.08228671 0.09789407 0.19455333 0.10349657 0.14737599
 0.11547743 0.1650914 ]
Table features: [50 12 55  7  5]
Combined input: [ 0.09382451  0.08228671  0.09789407  0.19455333  0.10349657  0.14737599
  0.11547743  0.1650914  50.         12.         55.          7.
  5.        ]
Label: fillna

Sequence: ['dropna', 'fillna'] → RNN probs: [0.09498138 0.08193863 0.09485827 0.19452812 0.10419852 0.1496172
 0.11462906 0.16524892]
Table features: [50 12 55  7  5]
Combined input: [ 0.09498138  0.08193863  0.09485827  0.19452812  0.10419852  0.1496172
  0.11462906  0.16524892 50.         12.         55.          7.
  5.        ]
Label: groupby

Sequence: ['dropna', 'fillna', 'groupby'] → RNN probs: [0.10210914 0.08281415 0.09465466 0.18479133 0.10357642 0.15474169
 0.11238545 0.16492726]
Table features: [50 12 55  7  5]
Combined input: [ 0.10210914  0.08281415  0.09465466  0.18479133  0.10357642  0.15474169
  0.11238545  0.16492726 50.         12.

In [ ]:
# Train the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical

X_final_np = np.array(X_final)
y_final_cat = to_categorical(y_final, num_classes=vocab_size) # transform labels to numbers
print(f"X shape: {X_final_np.shape}")
print(f"y shape: {y_final_cat.shape}")

final_model = Sequential()
final_model.add(Input(shape=(X_final_np.shape[1],)))
final_model.add(Dense(64, activation='relu'))
final_model.add(Dense(32, activation='relu'))
final_model.add(Dense(vocab_size, activation='softmax'))

final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
final_model.summary()

final_model.fit(X_final_np, y_final_cat, epochs=30, verbose=1)

X shape: (20, 13)
y shape: (20, 8)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,240 (12.66 KB)

 Trainable params: 3,240 (12.66 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1000 - loss: 16.3194
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.1000 - loss: 14.7435
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.2000 - loss: 13.6762
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.2000 - loss: 12.8630
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.2000 - loss: 12.0736
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.2000 - loss: 11.2906
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.2000 - loss: 10.4914
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.2000 - loss: 9.6690
Epoch 9/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.2000 - loss: 8.8391
Epoch 10/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.2000 - loss: 8.0193
Epoch 11/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.2000 - loss: 7.2340
Epoch 12/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.2000 - loss: 6.5

In [ ]:
# Predict next operator given a sequence + table
def predict_next_op_full(seq_ops, table_df):

  # We bring the sequence into the form
  seq_ids = le_rnn.transform(seq_ops)
  padded = pad_sequences([seq_ids], maxlen=max_len, padding='pre')

  # Get the prediction from rnn
  rnn_probs = model.predict(padded, verbose=0)[0]

  # Combine the rnn output with table's feattures
  table_feats = extract_table_features(table_df)
  combined = np.concatenate([rnn_probs, table_feats]).reshape(1, -1)


  # Use the combined X to make the final prediction
  pred = final_model.predict(combined, verbose=0)
  pred_op = le_rnn.inverse_transform([np.argmax(pred)])[0]
  return pred_op

# Example
test_ops = ['dropna', 'groupby']
predicted = predict_next_op_full(test_ops, df.head(100))
print(f"Sequence: {test_ops} → Predicted next operator: {predicted}")

Sequence: ['dropna', 'groupby'] → Predicted next operator: groupby


### Final Combined Model (RNN + Single-Operator Predictors)

In [ ]:
# Final Next-Operator Prediction Model: RNN + Single-Operator Predictors (Section 5)
#
# In this final version, we follow the paper’s design closely:
#
# We use the pretrained single-operator models from Section 4 — one for each of:
#   - merge, groupby, pivot, unpivot
#
# For a given step and a table Ti in the notebook:
# - We run each of these 4 operation-models (merge, groupby, pivot, unpivot) on the current table Ti.
# - Each model predicts how likely it is that Ti would be used as input
#   to that specific operation (e.g., "Is Ti ready for a groupby?").
#   - More specifically, for each operation-model , we:
#       • Extract the relevant features from Ti (same features used during training for all operators)
#       • Compute the predicted probability that Ti is suitable for that operation (by passing the relevant features into the
#         corresponding pretrained classifier's predict_proba())
#
# - The result is a 4-dimensional vector:
#     [P(merge | Ti), P(groupby | Ti), P(pivot | Ti), P(unpivot | Ti)]
#
# - This vector represents the "readiness" of the current table for each operation.
#   We concatenate it with the RNN output vector (operator sequence context),
#   and use this combined vector as input to a Multi-Layer Perceptron (MLP).
#
# Result:
# The MLP predicts the next operator by considering both the prior workflow and the current table’s structure.

In [ ]:
# Feature extractors used by pretrained single-operator models used above (this is just for simulating the final combined predictor)

def extract_groupby_features(df, col):
    feats = {}
    feats['distinct_count'] = df[col].nunique()
    feats['distinct_ratio'] = feats['distinct_count'] / len(df)
    feats['is_numeric'] = int(pd.api.types.is_numeric_dtype(df[col]))
    feats['null_ratio'] = df[col].isnull().sum() / len(df)
    return feats

def extract_unpivot_features(df, col):
    feats = {}
    feats['is_numeric'] = int(pd.api.types.is_numeric_dtype(df[col]))
    feats['distinct_ratio'] = df[col].nunique() / len(df)
    feats['null_ratio'] = df[col].isnull().sum() / len(df)
    value_counts = df[col].value_counts(dropna=True)
    feats['peak_ratio'] = value_counts.iloc[0] / len(df) if not value_counts.empty else 0
    name = col.lower()
    feats['name_is_metric'] = int(any(term in name for term in ['score', 'value', 'rate', 'amount', 'total', 'fare', 'age']))
    return feats

def extract_pivot_features(df):
    feats = {}
    feats['num_rows'] = len(df)
    feats['num_columns'] = df.shape[1]
    feats['num_nulls'] = df.isnull().sum().sum()
    feats['numeric_cols'] = df.select_dtypes(include='number').shape[1]
    return feats

def extract_join_features(df):
    feats = {}
    feats['num_rows'] = len(df)
    feats['num_columns'] = df.shape[1]
    feats['null_ratio'] = df.isnull().sum().sum() / (df.shape[0] * df.shape[1])
    feats['unique_vals_total'] = sum(df[col].nunique() for col in df.columns)
    return feats

In [ ]:
# Normally, the following operator-specific classifiers would be trained using real extracted features.
# For this small-scale simulation, we define quick placeholder models trained on simple synthetic feature vectors so that the final MLP pipeline runs.

from sklearn.linear_model import LogisticRegression
import numpy as np

# Dummy training data (just to make models callable)
X_dummy = np.random.rand(10, 4)
y_dummy = np.random.randint(0, 2, size=10)

join_clf = LogisticRegression().fit(X_dummy, y_dummy)
groupby_clf = LogisticRegression().fit(np.random.rand(10, 4), np.random.randint(0, 2, 10))
pivot_clf = LogisticRegression().fit(np.random.rand(10, 4), np.random.randint(0, 2, 10))
unpivot_clf = LogisticRegression().fit(np.random.rand(10, 5), np.random.randint(0, 2, 10))

In [ ]:
# Simulate a single-operator prediction using pretrained models (already trained previously)
def get_table_context_vector(df):
  features = []

  # Extract features and get prediction probability for each model
  join_feats = extract_join_features(df)
  features.append(join_clf.predict_proba([list(join_feats.values())])[0][1])  # P(join)

  groupby_feats = extract_groupby_features(df, df.columns[0])  # just one column as example
  features.append(groupby_clf.predict_proba([list(groupby_feats.values())])[0][1])  # P(groupby)

  pivot_feats = extract_pivot_features(df)  # use your pivot feature function
  features.append(pivot_clf.predict_proba([list(pivot_feats.values())])[0][1])  # P(pivot)

  unpivot_feats = extract_unpivot_features(df, df.columns[0])  # one column
  features.append(unpivot_clf.predict_proba([list(unpivot_feats.values())])[0][1])  # P(unpivot)

  return np.array(features)  # shape: (4,)

In [ ]:
# Combine RNN with table context for final prediction
X_final = []
y_final = []

for x_seq, y_op in zip(train_X, train_y):
  try:
    # Sequence context (RNN output)
    encoded = le_rnn.transform(x_seq)
    padded = pad_sequences([encoded], maxlen=max_len, padding='pre')
    rnn_probs = model.predict(padded, verbose=0)[0] # model is already trained

    # Table context vector from pretrained operator models
    table_probs = get_table_context_vector(df.head(50))  # simulate partial table
    combined = np.concatenate([rnn_probs, table_probs])

    # Combine and store
    X_final.append(combined)
    y_final.append(le_rnn.transform([y_op])[0]) # We don't forget to bring string labels into numbers
  except Exception:
    continue

In [ ]:
# Build and Train the final MLP
X_final_np = np.array(X_final)
y_final_cat = to_categorical(y_final, num_classes=vocab_size)# one-hot encoded vectors

# print("X_final_np shape:", X_final_np.shape)
# print("y_final length:", len(y_final))

# Create a simple MLP
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

final_model = Sequential()
final_model.add(Input(shape=(X_final_np.shape[1],)))
final_model.add(Dense(64, activation='relu'))
final_model.add(Dense(32, activation='relu'))
final_model.add(Dense(vocab_size, activation='softmax'))

final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the final model
final_model.fit(X_final_np, y_final_cat, epochs=30, verbose=1)

Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1000 - loss: 2.1724
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.1000 - loss: 2.1600
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.1000 - loss: 2.1481
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1000 - loss: 2.1365
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.1000 - loss: 2.1254
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.1000 - loss: 2.1146
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.1000 - loss: 2.1047
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.1000 - loss: 2.0961
Epoch 9/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.1000 - loss: 2.0880
Epoch 10/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1000 - loss: 2.0803
Epoch 11/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1000 - loss: 2.0735
Epoch 12/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.1000 - loss: 2.0669
Epo

In [ ]:
def predict_next_op_with_operator_models(seq_ops, table_df):

  # Bring sequence to the form and take the prediction from rnn
  rnn_ids = le_rnn.transform(seq_ops)
  padded = pad_sequences([rnn_ids], maxlen=max_len, padding='pre')
  rnn_probs = model.predict(padded, verbose=0)[0]

  # Take all probs from single predictors and combine them with the ones from rnn
  table_context = get_table_context_vector(table_df)
  combined = np.concatenate([rnn_probs, table_context]).reshape(1, -1)

  # Make the final prediction using the combined data calculated above
  pred = final_model.predict(combined, verbose=0)

  return le_rnn.inverse_transform([np.argmax(pred)])[0] # take max prob result and bring it to its initial form (not numeric)

# Example use:
history = ['dropna', 'groupby']
predicted = predict_next_op_with_operator_models(history, df.head(100))
print(f"Given history {history} → Predicted next operator: {predicted}")

Given history ['dropna', 'groupby'] → Predicted next operator: groupby
